# 0. Setup Paths and Dependencies 


In [10]:
WORKSPACE_PATH = 'workspace'
SCRIPTS_PATH = 'scripts'
APIMODEL_PATH = 'models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
MODEL_PATH = WORKSPACE_PATH+'/models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
IMAGE_PATH = WORKSPACE_PATH+'/images'
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [11]:
import tensorflow as tf
from object_detection.utils import config_util
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import numpy as np 
from PIL import Image
import pathlib
import cv2 
import matplotlib.pyplot as plt 

# 1. Load Train Model From Checkpoint

In [12]:
# Select model you want to use for detection 
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [13]:
CHECKPOINT_PATH = MODEL_PATH+'/{}/'.format(CUSTOM_MODEL_NAME)
CHECKPOINT_PATH

'workspace/models/my_ssd_mobnet/'

In [14]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-13')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [15]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

### Bash Command for Retrieving Model Metrics 

In [5]:
# Run this output in a bash terminal in the root directory to get the performance metrics 
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --checkpoint_dir={}/{}
""".format(APIMODEL_PATH, MODEL_PATH, CUSTOM_MODEL_NAME, MODEL_PATH, CUSTOM_MODEL_NAME, MODEL_PATH, CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_ssd_mobnet



# 2. Detect on Images 

In [7]:
# Number of steps you will train 
STEPS = '10000'
# Image you want to test 
IMAGE = 'guntest2'
# Path for the image you want to test 
TEST_IMAGE_PATH = '{}/{}.png'.format(IMAGE_PATH, IMAGE)
# Path and 
OUTPUT_PATH = '{}/{}_{}_steps_{}_result.png'.format(IMAGE_PATH, IMAGE, STEPS, CUSTOM_MODEL_NAME)

In [8]:
# Picking how the detected image is displayed 
# %matplotlib inline
plt.switch_backend('TkAgg')

In [13]:
img = cv2.imread(TEST_IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype = tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
             for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes']+label_id_offset,
    detections['detection_scores'], 
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=2,
    min_score_thresh=.6,
    agnostic_mode=False)

print(detections['detection_scores'][0])
print(detections['detection_scores'][1])
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.savefig(OUTPUT_PATH)
plt.show()

0.7648075
0.21023057


# 3. Detect in Real-Time and Videos 

In [20]:
# Setup for video capture
cap = cv2.VideoCapture(IMAGE_PATH + '/KeanuTestShort.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('your_video.avi', fourcc, 20.0, size)

In [ ]:
# Setup for live detection with webcam 
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [21]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=3,
                min_score_thresh=.6,
                agnostic_mode=False)

    # For video files and saves them 
    if ret == True:
        # Saves for video
        out.write(image_np_with_detections)

        # Display the resulting frame
        cv2.imshow('object detection', cv2.resize(image_np_with_detections, (1280, 720))) #(800, 600)))

        # Close window when "Q" button pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            break

    # For live feed 
#     cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (1280, 720))) #(800, 600)))
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         cap.release()
#         break

        
cap.release()
cap.release()
cv2.destroyAllWindows()

In [9]:
# stops detection 
cap.release()
cv2.destroyAllWindows()